In [1]:
#data source https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
TRAIN_DATA = "../data/external/train.csv"
TEST_DATA = "../data/external/test.csv"

#Target column in the data. 
TARGET = "SalePrice"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#display all dataframe columns
pd.options.display.max_columns = None

In [3]:
train = pd.read_csv(TRAIN_DATA)

In [4]:
train.drop('Id', axis=1, inplace=True)

### Variable Types

In [5]:
tmp = train.dtypes
vars_cat = tmp[tmp == 'O'].index.tolist()
vars_num = [col for col in tmp.index if col not in vars_cat and col!= "SalePrice"]
vars_num.remove('MoSold')
vars_num.remove('MSSubClass')
vars_cat.extend(['MoSold', 'MSSubClass'])

vars_year = [var for var in train.columns if 'Yr' in var or 'Year' in var]
#vars_num = [var for var in vars_num if var not in vars_year]

print(len(vars_year))
print(len(vars_num))
print(len(vars_cat))

4
34
45


### Year Variables

In [6]:
# modify year variables.
def modify_year_vars(df, var, var_ref="YrSold"):
    tmp = df[var_ref] - df[var] 
    return tmp

In [7]:
var_ref = 'YrSold'
for var in vars_year:
    if var != var_ref:
        train[var] = modify_year_vars(train, var)
        
train[vars_year].head()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,5,5,5.0,2008
1,31,31,31.0,2007
2,7,6,7.0,2008
3,91,36,8.0,2006
4,8,8,8.0,2008


### Numerical Variables

In [8]:
#numerical varialbes with missing values
# percentage of missing values
tmp = train[vars_num].isnull().mean().sort_values(ascending=False)
tmp[tmp>0]

LotFrontage    0.177397
GarageYrBlt    0.055479
MasVnrArea     0.005479
dtype: float64

In [9]:
tmp = train[vars_num].isnull().mean().sort_values(ascending=False)
vars_num_missing = tmp[tmp>0].index.tolist()

#adding a new column flagging missing values 
# fill the missing values in the original column with mean values
for var in vars_num_missing:
    tmp = np.where(train[var].isna(), 1, 0) #(1=missing, 0=not missing)
    train[var + '_na'] = tmp
    
    mean = train[var].mean()
    train[var] = train[var].fillna(mean)  # fill missing values with mean value

In [10]:
# checking the new flags
#they should add up to original percentage above.
for var in vars_num_missing:
    print(var + "_na", round(train[var + '_na'].mean(), 6))

LotFrontage_na 0.177397
GarageYrBlt_na 0.055479
MasVnrArea_na 0.005479


In [11]:
#checking again for missing values in the original variables
# there should not be any missing value at this point. 
tmp = train[vars_num].isnull().mean()
tmp[tmp>0]

Series([], dtype: float64)

### Categorical Values

In [12]:
print("Percentage of NA values among categorical variables:")
tmp = train[vars_cat].isnull().mean() 
vars_cat_with_na = tmp[tmp>0].index
print(tmp[tmp>0].sort_values(ascending=False)); print("")

vars_cat_missing_many = tmp[tmp >= 0.1].index.tolist()
vars_cat_missing_few = [var for var in tmp[tmp>0].index.tolist() if var not in vars_cat_missing_many]
print("Missing a large number of values:")
print(vars_cat_missing_many)
print("\nMissing fewer number of values:")
print(vars_cat_missing_few)

Percentage of NA values among categorical variables:
PoolQC          0.995205
MiscFeature     0.963014
Alley           0.937671
Fence           0.807534
FireplaceQu     0.472603
GarageType      0.055479
GarageFinish    0.055479
GarageQual      0.055479
GarageCond      0.055479
BsmtExposure    0.026027
BsmtFinType2    0.026027
BsmtQual        0.025342
BsmtCond        0.025342
BsmtFinType1    0.025342
MasVnrType      0.005479
Electrical      0.000685
dtype: float64

Missing a large number of values:
['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

Missing fewer number of values:
['MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']


In [13]:
for var in vars_cat_missing_few:
    mode = train[var].mode()[0]
    train[var] = train[var].fillna(mode)
    
for var in vars_cat_missing_many:
    train[var] = train[var].fillna("Missing")

#total number of missing categorical values
# expecting 0 values missing at this point
train[vars_cat].isnull().sum().sum()

0

In [14]:
vars_map_qual = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 
 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 
 'GarageQual', 'GarageCond', 'PoolQC']

map_qual ={
    'NA': 0,
    'Fa': 1,
    'TA': 2,
    'Gd': 3,
    'Ex': 4
}

for var in vars_map_qual:
    train[var] = train[var].map(map_qual)

##### To do

1. Cat - fill with "Missing" if there are too many missing values. 
       - fill with most frequent (mode) if there are small % of missing values. 
2. Num - new feature indicating missing
       - fill with mean
3. Time - 
4. Num - transform
5. Cat - map
6. Rare labels
7. Encode cat variables
8. Target - log
9. drop Id, year_sold
10. Scale (min max)
11. save